In [1]:
import json
import re

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm
from textblob import TextBlob

In [2]:
with open('../dataset/goodreads_reviews_spoiler.json') as json_file:
    data = json.loads('[' + ',\n'.join(json_file.readlines()) + ']')

# Preprocess

In [3]:
def unpack(sents):
    ys = []
    xs = []
    for y, sent in sents:
        ys.append(y)
        xs.append(sent)
    return ys, xs

In [4]:
x = []
y = []
for sample in tqdm(data):
    ans, sent = unpack(sample['review_sentences'])
    x.extend(sent)
    y.extend(ans)

100%|██████████| 1378033/1378033 [00:03<00:00, 405316.66it/s]


In [5]:
del data

In [6]:
for id, i in tqdm(enumerate(x), total=len(x)):
    text = re.sub(r'http\S+', '', i.lower())
    # text = str(TextBlob(text).correct())
    x[id] = text

100%|██████████| 17672655/17672655 [00:29<00:00, 601761.71it/s]


In [7]:
prep_data = pd.DataFrame([x,y], index=['text', 'target']).T

In [8]:
prep_data = prep_data[~(prep_data['text'] == '')]

In [40]:
prep_data = prep_data[prep_data.apply(lambda x: len(x[0]) <= 500, axis=1)]

In [46]:
(17659023 - len(prep_data))/17659023

6.466948935963219e-05

In [47]:
prep_data['text'] = prep_data['text'].apply(lambda x: np.str_(x))

/tmp/ipykernel_4820/2054751173.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prep_data['text'] = prep_data['text'].apply(lambda x: np.str_(x))


In [48]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(prep_data, test_size=0.2)

In [49]:
train.to_csv('dataset/train.csv', index=False)
test.to_csv('dataset/test.csv', index=False)

# TF-IDF baseline

In [1]:
import text_clf

In [2]:
model, target_names_mapping = text_clf.train(
                path_to_config='baseline_config_grid.yaml')

text-clf-train - INFO - Config:

seed: 42
path_to_save_folder: models
experiment_name: model

# data
data:
  train_data_path: dataset/train.csv
  test_data_path: dataset/test.csv
  sep: ','
  text_column: text
  target_column: target

# preprocessing
# (included in resulting model pipeline, so preserved for inference)
preprocessing:
  lemmatization: null  # pymorphy2

# tf-idf
tf-idf:
  lowercase: true
  ngram_range: (1, 1)
  max_df: 1.0
  min_df: 1

# logreg
logreg:
  penalty: l2
  C: 1.0
  class_weight: balanced
  solver: saga
  n_jobs: -1

# grid-search
grid-search:
  do_grid_search: true
  grid_search_params_path: hyperparams.py
text-clf-train - INFO - Loading data...
text-clf-train - INFO - Train dataset size: 14126304
text-clf-train - INFO - Test dataset size: 3531577
text-clf-train - INFO - Finding best hyper-parameters...
Fitting 3 folds for each of 8 candidates, totalling 24 fits
